In [2]:
# dependencies
!pip install mtcnn face_recognition tensorflow gradio

  Obtaining dependency information for face_recognition from https://files.pythonhosted.org/packages/1e/95/f6c9330f54ab07bfa032bf3715c12455a381083125d8880c43cbe76bb3d0/face_recognition-1.3.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/c5/49/ea7450fb03b926110181b758ff30a8426a6aaa60ae99f149ec3e7bacf04e/gradio-4.40.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     -------------------------------------- 0.0/100.1 MB 217.9 kB/s eta 0:07:40
     -------------------------------------- 0.0/100.1 MB 245.8 kB/s eta 0:06:48
     -------------------------------------- 0.1/100.1 MB 351.4 kB/s eta 0:04:45
     -------------------------------------- 0.1/100.1 MB 479.3 kB/s eta 0:03:29
     -------------------------------------

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Nas\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Nas\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Nas\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Nas\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Nas\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^

     -- ----------------------------------- 6.3/100.1 MB 163.5 kB/s eta 0:09:35
     -- ----------------------------------- 6.3/100.1 MB 164.0 kB/s eta 0:09:33
     -- ----------------------------------- 6.3/100.1 MB 164.4 kB/s eta 0:09:31
     -- ----------------------------------- 6.4/100.1 MB 165.4 kB/s eta 0:09:27
     -- ----------------------------------- 6.4/100.1 MB 166.6 kB/s eta 0:09:23
     -- ----------------------------------- 6.4/100.1 MB 166.6 kB/s eta 0:09:23
     -- ----------------------------------- 6.5/100.1 MB 167.1 kB/s eta 0:09:21
     -- ----------------------------------- 6.5/100.1 MB 167.7 kB/s eta 0:09:19
     -- ----------------------------------- 6.5/100.1 MB 169.1 kB/s eta 0:09:14
     -- ----------------------------------- 6.5/100.1 MB 169.1 kB/s eta 0:09:14
     -- ----------------------------------- 6.5/100.1 MB 169.1 kB/s eta 0:09:14
     -- ----------------------------------- 6.5/100.1 MB 169.1 kB/s eta 0:09:14
     -- --------------------------------

In [3]:
from mtcnn import MTCNN
from PIL import Image
import numpy as np
import face_recognition
import gradio as gr

import tensorflow as tf
import tensorflow.keras.model import Model
import tensorflow.layers import Input, Dense, Lambda
from tensorflow.keras.optimizers import Adam

SyntaxError: invalid syntax (397311763.py, line 8)

In [ ]:
def load_image(image_path):
    image = Image.open(image_path)
    image = Image.convert('RGB')
    return np.array(image)

In [ ]:
def detect_faces(image_array):
    detector = MTCNN()
    return detector.detec_faces(image_array)

In [ ]:
def get_face_embeddings(image_array, faces):
    encodings = []
    for face in faces:
        x, y, width, height = face['box']
        face_image = image_array[y:y+height, x:x+width]
        face_image = face_recognition.face_rescale(face_image, (180, 180))
        encoding = face_recognition.face_encodings(face_image)
        if encoding:
            encodings.append(encoding[0])
    return encodings

In [ ]:
def create_siamese_network(input_shape):
    input = Input(shape=input_shape)
    x = Dense(128, activation='relu')(input)
    x = Dense(128, activation='relu')(x)
    x = Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(x)
    model = Model(inputs=input, outputs=x)
    return model

In [ ]:
def create_siamese_model(input_shape):
    base_model = create_siamese_network(input_shape)
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    encoding_a = base_model(input_a)
    encoding_b = base_model(input_b)
    distance = Lambda(lambda x: tf.sqrt(tf.reduce_sum(tf.square(x[0] - x[1]), axis=1, keepdims=True)))([encoding_a, encoding_b])
    model = Model(inputs=[input_a, input_b], outputs=distance)
    model.compile(optimizer=Adam(), loss='binary_crossentropy')
    return model

In [ ]:
# Example training function (for illustration purposes)
def train_siamese_model(model, known_face_encodings, labels):
    # Assume `known_face_encodings` is a list of embeddings and `labels` is a list of labels
    # This is a placeholder; you need to implement proper training with your data
    pass

In [ ]:
def recognize_face(image1_path, image2_path, model, known_encodings):
    image1 = load_image(image1_path)
    image2 = load_image(image2_path)
    faces1 = detect_faces(image1)
    faces2 = detect_faces(image2)
    encodings1 = get_face_embeddings(image1, faces1)
    encodings2 = get_face_embeddings(image2, faces2)

    if encodings1 and encodings2:
        # Compare all combinations of encodings from image1 and image2
        distances = []
        for encoding1 in encodings1:
            for encoding2 in encodings2:
                distance = model.predict([np.array([encoding1]), np.array([encoding2])])[0][0]
                distances.append(distance)
        min_distance = min(distances) if distances else float('inf')
        match = min_distance < 0.5  # Adjust the threshold based on your needs
        return "Match found!" if match else "No match found."
    return "No faces detected."

In [ ]:
input_shape = (180, 180)
model = create_siamese_model(input_shape)

In [ ]:
def gradio_interface(image1, image2):
    result = recognize_face(image1, image2, model, known_encodings=[])
    return result

gr.Interface(fn=gradio_interface, inputs=["image", "image"], outputs="text").launch()
